In [1]:
import pandas as pd
import torch

# Part 1: Predicting with Hardcoded Model

In [2]:
# Loading train.csv to DataFrame and constructing a trainX and trainY tensor
train_df = pd.read_csv("train.csv")
train_tensor = torch.tensor(train_df.values, dtype=torch.float64)

trainX = train_tensor[:, [0,9]]

trainY = train_tensor[:, 10]
trainY = trainY.reshape(1044, -1)

In [3]:
# Loading test.csv to DataFrame and constructing a testX and testY tensor
test_df = pd.read_csv("test.csv")
test_tensor = torch.tensor(test_df.values, dtype=torch.float64)

testX = test_tensor[:, :-1]

testY = test_tensor[:, 10]
testY = testY.reshape(348, -1)

In [4]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=testX.dtype)

In [5]:
if torch.cuda.is_available():
    testX = testX.to("cuda")
    coef = coef.to("cuda")
predicted_deaths = testX[0] @ coef
predicted_deaths

tensor([9.8440], dtype=torch.float64)

# Part 2: R^2 Score

In [13]:
# Creating a predictedY tensor
predictedY = testX @ coef

# Function to calculate the R^2 score for any vector
def r2_score(testY, predictedY):
    
    if torch.cuda.is_available():
        testY = testY.to("cuda")
        
    testY_average = torch.mean(testY) 
    SStot_difference = testY - testY_average
    SStot = torch.sum(SStot_difference * SStot_difference)
    SSreg_difference = testY - predictedY
    SSreg = torch.sum(SSreg_difference * SSreg_difference)
    
    improvement = SStot - SSreg
    rsquared_score = improvement / SStot
    return rsquared_score

r2_score(testY, predictedY)

tensor(0.4110, dtype=torch.float64)